In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import matplotlib.colors as mcolors
import shutil
import random
from tqdm import tqdm
from PIL import Image
from skimage.transform import resize
from skimage.metrics import structural_similarity as ssim

# 4 imagenes en 1 Array:

In [3]:
# Benign
pathNumpy = "E:/Numpys/Array/Solo_Benign_Malignant/Benign"
pathNumpySave = "E:/Numpys/Array/Solo_Benign_Malignant/Array4/Benign"
os.makedirs(pathNumpySave, exist_ok=True)
archivos = os.listdir(pathNumpy)

for archivo in tqdm(archivos, desc="Procesando archivos"):
    ruta_archivo = os.path.join(pathNumpy, archivo)
    array_cargado = np.load(ruta_archivo)
    imagenes_redimensionadas = [np.array(Image.fromarray(imagen).resize((720, 720))) for imagen in array_cargado]
    while len(imagenes_redimensionadas) < 4:
        imagenes_redimensionadas.append(np.zeros((720, 720), dtype=np.uint8))
    array_redimensionado = np.array(imagenes_redimensionadas)
    np.save(os.path.join(pathNumpySave, archivo), array_redimensionado)

Procesando archivos: 100%|██████████████████████████████████████████████████████████| 607/607 [00:03<00:00, 161.40it/s]


In [6]:
# Malignant
pathNumpy = "E:/Numpys/Array/Solo_Benign_Malignant/Cancer"
pathNumpySave = "E:/Numpys/Array/Solo_Benign_Malignant/Array4/Malignant"
os.makedirs(pathNumpySave, exist_ok=True)
archivos = os.listdir(pathNumpy)

for archivo in tqdm(archivos, desc="Procesando archivos"):
    ruta_archivo = os.path.join(pathNumpy, archivo)
    array_cargado = np.load(ruta_archivo)
    imagenes_redimensionadas = [np.array(Image.fromarray(imagen).resize((720, 720))) for imagen in array_cargado]
    while len(imagenes_redimensionadas) < 4:
        imagenes_redimensionadas.append(np.zeros((720, 720), dtype=np.uint8))
    array_redimensionado = np.array(imagenes_redimensionadas)
    np.save(os.path.join(pathNumpySave, archivo), array_redimensionado)


Procesando archivos: 100%|████████████████████████████████████████████████████████| 1399/1399 [00:09<00:00, 149.60it/s]


In [2]:
def seleccionar_imagenes(columna, filas_mismo_study_uid, boxes_df, pathNumpy):
    # Inicializar las imágenes seleccionadas con imágenes en negro de tamaño 720x720
    imagenes_seleccionadas = np.zeros((4, 720, 720), dtype=np.uint8)
    for i, row in filas_mismo_study_uid.iterrows():
        PatientID = row["PatientID"]
        StudyUID = row["StudyUID"]
        view = row["View"]
        nombre = f"{view}-{i}.npy"
        ruta = os.path.join(f"{pathNumpy}{columna}", nombre)
        #print(ruta)
        if os.path.exists(ruta):
            imagen_cargada = np.load(ruta)
            #print(columna)
            if columna == "/Normal" or columna == "/Actionable":
                longitud_primera_dimension = imagen_cargada.shape[0]
                indice_slice = int(longitud_primera_dimension * 0.40898656) # Es la medi de donde se encuentran los canceres
            else:
                # Usando indexación booleana para filtrar el DataFrame
                fila_deseada = boxes_df[(boxes_df['PatientID'] == PatientID) & (boxes_df['StudyUID'] == StudyUID)]
                fila_deseada = fila_deseada.iloc[0]
                indice_slice = fila_deseada['Slice']

            #print("Indice Slice:",indice_slice)
            # Verificar si el índice es válido
            if indice_slice < imagen_cargada.shape[0]:
                if view == 'lcc':
                    imagenes_seleccionadas[0] = imagen_cargada[indice_slice, :, :, 1]
                elif view == 'rcc':
                    imagenes_seleccionadas[1] = imagen_cargada[indice_slice, :, :, 1]
                elif view == 'lmlo':
                    imagenes_seleccionadas[2] = imagen_cargada[indice_slice, :, :, 1]
                elif view == 'rmlo':
                    imagenes_seleccionadas[3] = imagen_cargada[indice_slice, :, :, 1]
                
    return imagenes_seleccionadas

def obtener_columna(fila):
    # Verificar el valor de cada columna y asignar el nombre correspondiente a las variables
    columna = None
    if fila["Normal"] == 1:
        columna = "/Normal"
    elif fila["Actionable"] == 1:
        columna = "/Actionable"
    elif fila["Benign"] == 1:
        columna = "/Benign"
    elif fila["Cancer"] == 1:
        columna = "/Cancer"
    return columna

In [14]:
# Validation
patgdfBoxes = "D:/Codigo/Datos/InfValidation/boxes-validation.csv"
pathdf = "D:/Codigo/Datos/InfValidation/merged_dfValidation_serialized.pickle"
pathNumpySave = "E:/Numpys/DBT/TotalArraysImagenes"
pathNumpy = "D:/Numpys/DBT Totales/NumpysDBT/Total"

merged_df = pd.read_pickle(pathdf)
df_boxes = pd.read_csv(patgdfBoxes)

for index, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Cargando imagen"):
    filas_mismo_study_uid = merged_df[merged_df['StudyUID'] == row['StudyUID']]
    
    if not filas_mismo_study_uid.empty:
        merged_df.drop(filas_mismo_study_uid.index, inplace=True)
        
        columna = obtener_columna(filas_mismo_study_uid.iloc[0])
        imagenes_seleccionadas = seleccionar_imagenes(columna, filas_mismo_study_uid, df_boxes, pathNumpy)
        
        imagenes_redimensionadas = []
        for imagen in imagenes_seleccionadas:# Redimensionar las imágenes a 720x720 píxeles
            imagen_pil = Image.fromarray(imagen)
            imagen_redimensionada = imagen_pil.resize((720, 720))
            imagenes_redimensionadas.append(np.array(imagen_redimensionada))
        
        imagenes_redimensionadas_array = np.array(imagenes_redimensionadas)
        
        nombre = f"{filas_mismo_study_uid.iloc[0]['StudyUID']}_{filas_mismo_study_uid.iloc[0]['PatientID']}"
        ruta_archivo = os.path.join(f"{pathNumpySave}{columna}", nombre)
        if os.path.exists(ruta_archivo):
            print(f"El archivo {ruta_archivo} ya existe.")
        
        np.save(ruta_archivo, imagenes_redimensionadas_array)

Cargando imagen: 100%|█████████████████████████████████████████████████████████████| 1163/1163 [09:35<00:00,  2.02it/s]


In [16]:
# Training
patgdfBoxes = "D:/Codigo/Datos/InfTraining/boxes-train.csv"
pathdf = "D:/Codigo/Datos/InfTraining/merged_dfTraining_serialized.pickle"
pathNumpySave = "E:/Numpys/DBT/TotalArraysImagenes"
pathNumpy = "D:/Numpys/DBT Totales/NumpysDBT/Total"

merged_df = pd.read_pickle(pathdf)
df_boxes = pd.read_csv(patgdfBoxes)

for index, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Cargando imagen"):
    filas_mismo_study_uid = merged_df[merged_df['StudyUID'] == row['StudyUID']]
    
    if not filas_mismo_study_uid.empty:
        merged_df.drop(filas_mismo_study_uid.index, inplace=True)
        columna = obtener_columna(filas_mismo_study_uid.iloc[0])
        
        imagenes_seleccionadas = seleccionar_imagenes(columna, filas_mismo_study_uid, df_boxes, pathNumpy)
        
        imagenes_redimensionadas = []
        for imagen in imagenes_seleccionadas:# Redimensionar las imágenes a 720x720 píxeles
            imagen_pil = Image.fromarray(imagen)
            imagen_redimensionada = imagen_pil.resize((720, 720))
            imagenes_redimensionadas.append(np.array(imagen_redimensionada))
        
        imagenes_redimensionadas_array = np.array(imagenes_redimensionadas)
        
        nombre = f"{filas_mismo_study_uid.iloc[0]['StudyUID']}_{filas_mismo_study_uid.iloc[0]['PatientID']}"
        ruta_archivo = os.path.join(f"{pathNumpySave}{columna}", nombre)
        if os.path.exists(ruta_archivo):
            print(f"El archivo {ruta_archivo} ya existe.")
        
        np.save(ruta_archivo, imagenes_redimensionadas_array)

Cargando imagen: 100%|█████████████████████████████████████████████████████████| 19148/19148 [2:20:06<00:00,  2.28it/s]


In [3]:
# Test
patgdfBoxes = "D:/Codigo/Datos/InfTest/BCS-DBT-boxes-test-v2-PHASE-2-Jan-2024.csv"
pathdf = "D:/Codigo/Datos/InfTest/merged_dfTest_serialized.pickle"
pathNumpySave = "E:/Numpys/DBT/TotalArraysImagenes"
pathNumpy = "D:/Numpys/DBT Totales/NumpysDBT/Total"

merged_df = pd.read_pickle(pathdf)
df_boxes = pd.read_csv(patgdfBoxes)

for index, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Cargando imagen"):
    filas_mismo_study_uid = merged_df[merged_df['StudyUID'] == row['StudyUID']]
    
    if not filas_mismo_study_uid.empty:
        merged_df.drop(filas_mismo_study_uid.index, inplace=True)
        
        columna = obtener_columna(filas_mismo_study_uid.iloc[0])
        
        imagenes_seleccionadas = seleccionar_imagenes(columna, filas_mismo_study_uid, df_boxes, pathNumpy)
        
        imagenes_redimensionadas = []
        for imagen in imagenes_seleccionadas:# Redimensionar las imágenes a 720x720 píxeles
            imagen_pil = Image.fromarray(imagen)
            imagen_redimensionada = imagen_pil.resize((720, 720))
            imagenes_redimensionadas.append(np.array(imagen_redimensionada))
        
        imagenes_redimensionadas_array = np.array(imagenes_redimensionadas)
        
        nombre = f"{filas_mismo_study_uid.iloc[0]['StudyUID']}_{filas_mismo_study_uid.iloc[0]['PatientID']}"
        ruta_archivo = os.path.join(f"{pathNumpySave}{columna}", nombre)
        if os.path.exists(ruta_archivo):
            print(f"El archivo {ruta_archivo} ya existe.")
        
        np.save(ruta_archivo, imagenes_redimensionadas_array)

Cargando imagen: 100%|█████████████████████████████████████████████████████████████| 1721/1721 [14:56<00:00,  1.92it/s]


# ------------------------------ Comparar y seleccionar imágenes --------------------------------

In [4]:
# Benign
pathNumpy = "E:/Numpys/CMMD/Array/Benign"
pathNumpySave = "E:/Numpys/CMMD/CMMD_ArrayNumpysSeleccionadas/Benign"
NumImagenes = 280 - 112  # Img Cancer Validation dataset

# Lista para almacenar las rutas de los archivos .npy
rutas_archivos = []
for filename in os.listdir(pathNumpy):
    if filename.endswith(".npy"):
        rutas_archivos.append(os.path.join(pathNumpy, filename))

# Mostrar la cantidad de archivos en rutas_archivos
print("Cantidad de archivos .npy encontrados:", len(rutas_archivos))

archivo_progreso = "progresoCMMDSeleccionBenign.npy"
# Comprobar si existe un archivo de progreso guardado
if os.path.exists(archivo_progreso):
    print("Cargando progreso previo...")
    ssim_scores_promedio = np.load(archivo_progreso)
else:
    ssim_scores_promedio = np.empty((0, 2))

# Ordenar los scores promedio de menor a mayor
sorted_indices = np.argsort(ssim_scores_promedio[:, 1])

# Seleccionar los primeros NumImagenes
indices_seleccionados = sorted_indices[:NumImagenes]
print(indices_seleccionados)
# Guardar los archivos seleccionados con sus nombres originales
for idx in indices_seleccionados:
    nombre_original = os.path.basename(rutas_archivos[idx])  # Obtener el nombre original del archivo
    np.save(os.path.join(pathNumpySave, nombre_original), np.load(rutas_archivos[idx]))
print("Fin")

Cantidad de archivos .npy encontrados: 495
Cargando progreso previo...
[203 281 488 222 195  31  49 480  57 355 391  68 116 367  46 298 493 492
  44  85 311 368  51 312 487  47 375 489  43 313 344 228 252 486 481 328
  92 479 289 113 111 435 106   7 365 400 485  64 115 279 114 144 238 309
 200 482  48  33 426 310 340 427 256 181 483 374   8 220  70 416 442 251
  23 107 224 330 445 149  38 491 254  90 318 450 449  39 433 273  45 320
 264 214  29  74 152 299 242  52 239 265 390 353 117   4 270 237 366 409
 463 494 124 341 422 382 381 146 123 206   9 454 223  84 465 230 396 142
 257  65  15  53 484  62 447  86 178 490 119 284 438 110 315  34  61 434
 187 455 227 466 410  40 161 262  20 468 470 272 305 338 332 412  56 331
 472  91 198  17  22 415]
Fin


In [5]:
# Malignant
pathNumpy = "E:/Numpys/CMMD/Array/Malignant"
pathNumpySave = "E:/Numpys/CMMD/CMMD_ArrayNumpysSeleccionadas/Malignant"
NumImagenes = 280 - 89  # Img Cancer Validation dataset

# Lista para almacenar las rutas de los archivos .npy
rutas_archivos = []
for filename in os.listdir(pathNumpy):
    if filename.endswith(".npy"):
        rutas_archivos.append(os.path.join(pathNumpy, filename))

# Mostrar la cantidad de archivos en rutas_archivos
print("Cantidad de archivos .npy encontrados:", len(rutas_archivos))

archivo_progreso = "progresoCMMDSeleccionMalignant.npy"
# Comprobar si existe un archivo de progreso guardado
if os.path.exists(archivo_progreso):
    print("Cargando progreso previo...")
    ssim_scores_promedio = np.load(archivo_progreso)
else:
    ssim_scores_promedio = np.empty((0, 2))

# Ordenar los scores promedio de menor a mayor
sorted_indices = np.argsort(ssim_scores_promedio[:, 1])

# Seleccionar los primeros NumImagenes
indices_seleccionados = sorted_indices[:NumImagenes]
print(indices_seleccionados)
# Guardar los archivos seleccionados con sus nombres originales
for idx in indices_seleccionados:
    nombre_original = os.path.basename(rutas_archivos[idx])  # Obtener el nombre original del archivo
    np.save(os.path.join(pathNumpySave, nombre_original), np.load(rutas_archivos[idx]))
print("Fin")

Cantidad de archivos .npy encontrados: 1310
Cargando progreso previo...
[1174  928 1011 1231  793  975 1205 1201  916 1196  923  775  830  859
  519 1034 1308  789 1237  965  773  670 1304 1279  723  419 1019  652
 1143 1246  781 1210 1087  225  721  313 1059  398  137 1027 1190  963
 1015  601  448 1043  466  224 1032  672  784  803 1167 1121  371   80
  417 1060  379 1058  752  470  926 1219  758  246 1164  771  522 1077
  444 1033  968  953 1218  914  296 1126  537  915  805 1066  531  514
 1005  410  811 1056 1278  977  639   42 1175 1239  280  828  857 1170
  193  903 1130  489 1124 1031 1097  266  134  325  688  357  183  689
 1039   81 1008  510  429  427  547  320  919  779  342 1114 1263  428
  316  382   77  153  921  449 1302  286  307  396  538  375 1198 1045
  230 1064  493 1257  322  359  480  657  766  354    2   50  730  463
  986 1131  279 1020   76 1028  200  441  235  984  683 1075  189  712
  376  971 1197 1277  647  497   69  499  350  314  544  482  869  840
  161

In [ ]:
# Función para calcular la Diferencia Absoluta (MAE) entre dos imágenes
def calculate_mae(image1, image2):
    return np.mean(np.abs(image1 - image2))

In [4]:
# Normal
pathNumpy = "E:/Numpys/DBT/TotalArraysImagenes/Normal"
pathNumpySave = ""
NumImagenes = 400

rutas_archivos = []
for filename in os.listdir(pathNumpy):
    if filename.endswith(".npy"):
        rutas_archivos.append(os.path.join(pathNumpy, filename))

# Lista para almacenar las rutas de los archivos .npy
print("Cantidad de archivos .npy encontrados:", len(rutas_archivos))

# Seleccionar el 25% de las imágenes
rutas_archivos = rutas_archivos[:int(len(rutas_archivos) * 0.5)]

print("Cantidad de archivos .npy seleccionados:", len(rutas_archivos))

# Comprobar si existe un archivo de progreso guardado
archivo_progreso = "progresoSeleccionNormal2.npy"
if os.path.exists(archivo_progreso):
    print("Cargando progreso previo...")
    ssim_scores_promedio = np.load(archivo_progreso)
else:
    ssim_scores_promedio = np.empty((0,))

# Calcular la Diferencia Absoluta (MAE) entre cada par de imágenes si no se ha calculado previamente
for i, ruta_i in enumerate(tqdm(rutas_archivos, desc="Comparando imágenes", unit="archivo", leave=False)):
    if i >= len(ssim_scores_promedio) and i > :  # Continuar desde donde se quedó
        archivo_i = np.load(ruta_i)
        scores_i = []
        for j, ruta_j in enumerate(rutas_archivos):
            if i != j:  # Evitar comparar un archivo consigo mismo
                archivo_j = np.load(ruta_j)
                score = calculate_mae(archivo_i, archivo_j)
                scores_i.append(score)
        ssim_scores_promedio = np.append(ssim_scores_promedio, np.mean(scores_i))
        
        np.save(archivo_progreso, ssim_scores_promedio)

# print(ssim_scores_promedio.shape)
# # Ordenar los scores promedio de menor a mayor
# sorted_indices = np.argsort(ssim_scores_promedio)

# # Seleccionar los NumImagenes archivos menos similares al resto
# indices_seleccionados = sorted_indices[:NumImagenes]

# # Guardar los archivos seleccionados con sus nombres originales
# for idx in indices_seleccionados:
#     nombre_original = os.path.basename(rutas_archivos[idx])  # Obtener el nombre original del archivo
#     np.save(os.path.join(pathNumpySave, nombre_original), np.load(rutas_archivos[idx]))

Cantidad de archivos .npy encontrados: 5129
Cantidad de archivos .npy seleccionados: 1282
Cargando progreso previo...
(1282,)


# Dividir en bloques Training, Validation y Test

In [2]:
# Función para copiar archivos a los directorios correspondientes según el prefijo
def copy_files_by_prefix(prefixes, source_dir, output_dir):
    for prefix in prefixes:
        # Obtener los archivos correspondientes al prefijo
        prefix_files = [file for file in os.listdir(source_dir) if file.startswith(prefix)]
        
        # Copiar los archivos a los directorios correspondientes
        for file in prefix_files:
            source_file = os.path.join(source_dir, file)
            if prefix in training_prefixes:
                destination = os.path.join(output_dir, "Training", file)
            elif prefix in validation_prefixes:
                destination = os.path.join(output_dir, "Validation", file)
            else:
                destination = os.path.join(output_dir, "Test", file)
            shutil.copy(source_file, destination)

In [12]:
# Benign
input_directory = "E:/Numpys/Array/Solo_Benign_Malignant/Malignant"
output_directory = "E:/Numpys/Array/Solo_Benign_Malignant/Division"

# Crear directorios de Training, Validation y Test si no existen
for division in ["Training", "Validation", "Test"]:
    division_dir = os.path.join(output_directory, division)
    if not os.path.exists(division_dir):
        os.makedirs(division_dir)

# Obtener la lista de archivos npy en el directorio de entrada
npy_files = [f for f in os.listdir(input_directory) if f.endswith('.npy')]

# Crear una lista para almacenar los prefijos de las imágenes
image_prefixes = []
for npy_file in npy_files:
    #file_name = os.path.splitext(npy_file)[0]  # Obtener el nombre del archivo sin la extensión .npy
    image_prefixes.append(npy_file)

# Mostrar los prefijos guardados por pantalla
# print("Prefijos de las imágenes:")
# for prefix in image_prefixes:
#     print(prefix)

# Mostrar la cantidad total de prefijos
total_prefixes = len(image_prefixes)
print("Cantidad total de prefijos:", total_prefixes)

# Determinar la cantidad de prefijos para cada conjunto (Training, Validation, Test)
num_prefixes = len(image_prefixes)
num_test_prefixes = int(0.2 * num_prefixes) # 20% para test
num_training_prefixes = int(0.8 * (num_prefixes-num_test_prefixes))  # 80 del restante para Training
num_validation_prefixes =  num_prefixes-num_test_prefixes-num_training_prefixes # Validation

# Imprimir la cantidad de prefijos para cada conjunto
print("Cantidad de prefijos para Training:", num_training_prefixes)
print("Cantidad de prefijos para Validation:", num_validation_prefixes)
print("Cantidad de prefijos para Test:", num_test_prefixes)

# Seleccionar aleatoriamente los prefijos para cada conjunto
random.shuffle(image_prefixes)
training_prefixes = image_prefixes[:num_training_prefixes]
validation_prefixes = image_prefixes[num_training_prefixes:num_training_prefixes + num_validation_prefixes]
test_prefixes = image_prefixes[num_training_prefixes + num_validation_prefixes:]

# Mostrar los prefijos seleccionados para cada conjunto por pantalla
# print("Prefijos seleccionados para Training:")
# for prefix in training_prefixes:
#     print(prefix)
# print("Prefijos seleccionados para Validation:")
# for prefix in validation_prefixes:
#     print(prefix)
# print("Prefijos seleccionados para Test:")
# for prefix in test_prefixes:
#     print(prefix)

# Copiar archivos por prefijo para Training, Validation y Test
files_directory = os.path.join(output_directory, "Training/Malignant")
for prefix in training_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)

print("Training completado.")
files_directory = os.path.join(output_directory, "Validation/Malignant")
for prefix in validation_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)
    
print("Validation completado.")
files_directory = os.path.join(output_directory, "Test/Malignant")
for prefix in test_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)
    
print("Test completado.")

print("Proceso completado.")

Cantidad total de prefijos: 1399
Cantidad de prefijos para Training: 896
Cantidad de prefijos para Validation: 224
Cantidad de prefijos para Test: 279
Training completado.
Validation completado.
Test completado.
Proceso completado.


In [6]:
# Actionable
input_directory = "E:/Numpys/Array/Bloques 607 Benign malign/Division/Actionable"
output_directory = "E:/Numpys/Array/Bloques 607 Benign malign/Division"

# Crear directorios de Training, Validation y Test si no existen
for division in ["Training", "Validation", "Test"]:
    division_dir = os.path.join(output_directory, division)
    if not os.path.exists(division_dir):
        os.makedirs(division_dir)

# Obtener la lista de archivos npy en el directorio de entrada
npy_files = [f for f in os.listdir(input_directory) if f.endswith('.npy')]

# Crear una lista para almacenar los prefijos de las imágenes
image_prefixes = []
for npy_file in npy_files:
    #file_name = os.path.splitext(npy_file)[0]  # Obtener el nombre del archivo sin la extensión .npy
    image_prefixes.append(npy_file)

# Mostrar los prefijos guardados por pantalla
# print("Prefijos de las imágenes:")
# for prefix in image_prefixes:
#     print(prefix)

# Mostrar la cantidad total de prefijos
total_prefixes = len(image_prefixes)
print("Cantidad total de prefijos:", total_prefixes)

# Determinar la cantidad de prefijos para cada conjunto (Training, Validation, Test)
num_prefixes = len(image_prefixes)
num_training_prefixes = int(0.6 * num_prefixes)  # 70% para Training
num_validation_prefixes = int(0.2 * num_prefixes)  # 15% para Validation
num_test_prefixes = num_prefixes - num_training_prefixes - num_validation_prefixes  # El resto para Test

# Imprimir la cantidad de prefijos para cada conjunto
print("Cantidad de prefijos para Training:", num_training_prefixes)
print("Cantidad de prefijos para Validation:", num_validation_prefixes)
print("Cantidad de prefijos para Test:", num_test_prefixes)

# Seleccionar aleatoriamente los prefijos para cada conjunto
random.shuffle(image_prefixes)
training_prefixes = image_prefixes[:num_training_prefixes]
validation_prefixes = image_prefixes[num_training_prefixes:num_training_prefixes + num_validation_prefixes]
test_prefixes = image_prefixes[num_training_prefixes + num_validation_prefixes:]

# Mostrar los prefijos seleccionados para cada conjunto por pantalla
# print("Prefijos seleccionados para Training:")
# for prefix in training_prefixes:
#     print(prefix)
# print("Prefijos seleccionados para Validation:")
# for prefix in validation_prefixes:
#     print(prefix)
# print("Prefijos seleccionados para Test:")
# for prefix in test_prefixes:
#     print(prefix)

# Copiar archivos por prefijo para Training, Validation y Test
files_directory = os.path.join(output_directory, "Training/Actionable")
for prefix in training_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)

print("Training completado.")
files_directory = os.path.join(output_directory, "Validation/Actionable")
for prefix in validation_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)
    
print("Validation completado.")
files_directory = os.path.join(output_directory, "Test/Actionable")
for prefix in test_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)
    
print("Test completado.")

print("Proceso completado.")

Cantidad total de prefijos: 280
Cantidad de prefijos para Training: 168
Cantidad de prefijos para Validation: 56
Cantidad de prefijos para Test: 56
Training completado.
Validation completado.
Test completado.
Proceso completado.


In [4]:
# Cancer
input_directory = "E:/Numpys/Array/Bloques 607 Benign malign/Division/Cancer"
output_directory = "E:/Numpys/Array/Bloques 607 Benign malign/Division"

# Crear directorios de Training, Validation y Test si no existen
for division in ["Training", "Validation", "Test"]:
    division_dir = os.path.join(output_directory, division)
    if not os.path.exists(division_dir):
        os.makedirs(division_dir)

# Obtener la lista de archivos npy en el directorio de entrada
npy_files = [f for f in os.listdir(input_directory) if f.endswith('.npy')]

# Crear una lista para almacenar los prefijos de las imágenes
image_prefixes = []
for npy_file in npy_files:
    #file_name = os.path.splitext(npy_file)[0]  # Obtener el nombre del archivo sin la extensión .npy
    image_prefixes.append(npy_file)

# Mostrar los prefijos guardados por pantalla
# print("Prefijos de las imágenes:")
# for prefix in image_prefixes:
#     print(prefix)

# Mostrar la cantidad total de prefijos
total_prefixes = len(image_prefixes)
print("Cantidad total de prefijos:", total_prefixes)

# Determinar la cantidad de prefijos para cada conjunto (Training, Validation, Test)
num_prefixes = len(image_prefixes)
num_training_prefixes = int(0.6 * num_prefixes)  # 70% para Training
num_validation_prefixes = int(0.2 * num_prefixes)  # 15% para Validation
num_test_prefixes = num_prefixes - num_training_prefixes - num_validation_prefixes  # El resto para Test

# Imprimir la cantidad de prefijos para cada conjunto
print("Cantidad de prefijos para Training:", num_training_prefixes)
print("Cantidad de prefijos para Validation:", num_validation_prefixes)
print("Cantidad de prefijos para Test:", num_test_prefixes)

# Seleccionar aleatoriamente los prefijos para cada conjunto
random.shuffle(image_prefixes)
training_prefixes = image_prefixes[:num_training_prefixes]
validation_prefixes = image_prefixes[num_training_prefixes:num_training_prefixes + num_validation_prefixes]
test_prefixes = image_prefixes[num_training_prefixes + num_validation_prefixes:]

# Mostrar los prefijos seleccionados para cada conjunto por pantalla
# print("Prefijos seleccionados para Training:")
# for prefix in training_prefixes:
#     print(prefix)
# print("Prefijos seleccionados para Validation:")
# for prefix in validation_prefixes:
#     print(prefix)
# print("Prefijos seleccionados para Test:")
# for prefix in test_prefixes:
#     print(prefix)

# Copiar archivos por prefijo para Training, Validation y Test
files_directory = os.path.join(output_directory, "Training/Cancer")
for prefix in training_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)

print("Training completado.")
files_directory = os.path.join(output_directory, "Validation/Cancer")
for prefix in validation_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)
    
print("Validation completado.")
files_directory = os.path.join(output_directory, "Test/Cancer")
for prefix in test_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)
    
print("Test completado.")

print("Proceso completado.")

Cantidad total de prefijos: 1399
Cantidad de prefijos para Training: 839
Cantidad de prefijos para Validation: 279
Cantidad de prefijos para Test: 281
Training completado.
Validation completado.
Test completado.
Proceso completado.


In [8]:
# Normal
input_directory = "E:/Numpys/Array/Bloques 607 Benign malign/Division/Normal"
output_directory = "E:/Numpys/Array/Bloques 607 Benign malign/Division"

# Crear directorios de Training, Validation y Test si no existen
for division in ["Training", "Validation", "Test"]:
    division_dir = os.path.join(output_directory, division)
    if not os.path.exists(division_dir):
        os.makedirs(division_dir)

# Obtener la lista de archivos npy en el directorio de entrada
npy_files = [f for f in os.listdir(input_directory) if f.endswith('.npy')]

# Crear una lista para almacenar los prefijos de las imágenes
image_prefixes = []
for npy_file in npy_files:
    #file_name = os.path.splitext(npy_file)[0]  # Obtener el nombre del archivo sin la extensión .npy
    image_prefixes.append(npy_file)

# Mostrar los prefijos guardados por pantalla
# print("Prefijos de las imágenes:")
# for prefix in image_prefixes:
#     print(prefix)

# Mostrar la cantidad total de prefijos
total_prefixes = len(image_prefixes)
print("Cantidad total de prefijos:", total_prefixes)

# Determinar la cantidad de prefijos para cada conjunto (Training, Validation, Test)
num_prefixes = len(image_prefixes)
num_training_prefixes = int(0.6 * num_prefixes)  # 70% para Training
num_validation_prefixes = int(0.2 * num_prefixes)  # 15% para Validation
num_test_prefixes = num_prefixes - num_training_prefixes - num_validation_prefixes  # El resto para Test

# Imprimir la cantidad de prefijos para cada conjunto
print("Cantidad de prefijos para Training:", num_training_prefixes)
print("Cantidad de prefijos para Validation:", num_validation_prefixes)
print("Cantidad de prefijos para Test:", num_test_prefixes)

# Seleccionar aleatoriamente los prefijos para cada conjunto
random.shuffle(image_prefixes)
training_prefixes = image_prefixes[:num_training_prefixes]
validation_prefixes = image_prefixes[num_training_prefixes:num_training_prefixes + num_validation_prefixes]
test_prefixes = image_prefixes[num_training_prefixes + num_validation_prefixes:]

# Mostrar los prefijos seleccionados para cada conjunto por pantalla
# print("Prefijos seleccionados para Training:")
# for prefix in training_prefixes:
#     print(prefix)
# print("Prefijos seleccionados para Validation:")
# for prefix in validation_prefixes:
#     print(prefix)
# print("Prefijos seleccionados para Test:")
# for prefix in test_prefixes:
#     print(prefix)

# Copiar archivos por prefijo para Training, Validation y Test
files_directory = os.path.join(output_directory, "Training/Normal")
for prefix in training_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)

print("Training completado.")
files_directory = os.path.join(output_directory, "Validation/Normal")
for prefix in validation_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)
    
print("Validation completado.")
files_directory = os.path.join(output_directory, "Test/Normal")
for prefix in test_prefixes:
    source_file = os.path.join(input_directory, prefix)
    destination = os.path.join(files_directory, prefix)
    shutil.copy(source_file, destination)
    
print("Test completado.")

print("Proceso completado.")

Cantidad total de prefijos: 1000
Cantidad de prefijos para Training: 600
Cantidad de prefijos para Validation: 200
Cantidad de prefijos para Test: 200
Training completado.
Validation completado.
Test completado.
Proceso completado.


# Data Augmentation

In [2]:
def flip_image_horizontal(image):
    """
    Flips an image horizontally.
    """
    flipped_image = cv2.flip(image, 1)
    return flipped_image

def flip_image_vertical(image):
    """
    Flips an image vertically.
    """
    flipped_image = cv2.flip(image, 0)
    return flipped_image
def rotate_image_90(image, times=1):
    """
    Rotates an image by 90 degrees clockwise.
    """
    rotated_image = image.copy()
    for _ in range(times):
        rotated_image = cv2.rotate(rotated_image, cv2.ROTATE_90_CLOCKWISE)
    return rotated_image

In [10]:
# Ejemplo de uso:
# Cargar el archivo .npy
file_path = 'E:/Numpys/Array/Bloques 607 Benign malign/Division/Test/Normal/DBT-S02802_DBT-P04070.npy'
images = np.load(file_path)

# Generar todas las combinaciones posibles de transformaciones
transformations = [
    (0, 0, 0),  # No transformation
    (1, 0, 0),  # Rotación de 90 grados
    (2, 0, 0),  # Rotación de 180 grados
    (3, 0, 0),  # Rotación de 270 grados
    (0, 1, 0),  # Volteo horizontal
    (0, 0, 1),  # Volteo vertical
    (1, 1, 0),  # Rotación de 90 grados + Volteo horizontal
    (1, 0, 1),  # Rotación de 90 grados + Volteo vertical
    (2, 1, 0),  # Rotación de 180 grados + Volteo horizontal
    (2, 0, 1),  # Rotación de 180 grados + Volteo vertical
    (3, 1, 0),  # Rotación de 270 grados + Volteo horizontal
    (3, 0, 1),  # Rotación de 270 grados + Volteo vertical
]

# Aplicar las transformaciones a cada imagen y guardar en un nuevo archivo .npy
for index, original_image in enumerate(images):
    for rotation, horizontal_flip, vertical_flip in transformations:
        transformed_image = original_image.copy()
        transformed_image = rotate_image_90(transformed_image, times=rotation)
        if horizontal_flip:
            transformed_image = flip_image_horizontal(transformed_image)
        if vertical_flip:
            transformed_image = flip_image_vertical(transformed_image)
    
        # Guardar las imágenes transformadas en un nuevo archivo .npy
        output_file_path = f'resultados/transformada/imagen_{index}_{rotation}{horizontal_flip}{vertical_flip}.jpg'
        cv2.imwrite(output_file_path, transformed_image)
print(f'Imágenes transformadas de la imagen {index} guardadas en resultados/transformada/imagen')

Imágenes transformadas de la imagen 3 guardadas en resultados/transformada/imagen


In [3]:
# Test:
pathNumpy = "E:/Numpys/Array/Bloques 607 Benign malign/Division/Test"
pathNumpySave = "E:/Numpys/Array/Bloques 607 Benign malign/DataAugmentation/Test"

transformations = [
    (0, 0, 0),  # No transformation
    (1, 0, 0),  # Rotación de 90 grados
    (2, 0, 0),  # Rotación de 180 grados
    (3, 0, 0),  # Rotación de 270 grados
    (0, 1, 0),  # Volteo horizontal
    (0, 0, 1),  # Volteo vertical
    (1, 1, 0),  # Rotación de 90 grados + Volteo horizontal
    (1, 0, 1),  # Rotación de 90 grados + Volteo vertical
    (2, 1, 0),  # Rotación de 180 grados + Volteo horizontal
    (2, 0, 1),  # Rotación de 180 grados + Volteo vertical
    (3, 1, 0),  # Rotación de 270 grados + Volteo horizontal
    (3, 0, 1),  # Rotación de 270 grados + Volteo vertical
]

contenido = os.listdir(pathNumpy)
carpetas = [elemento for elemento in contenido if os.path.isdir(os.path.join(pathNumpy, elemento))]
for carpeta in carpetas:
    for filename in os.listdir(os.path.join(pathNumpy,carpeta)):
        if filename.endswith(".npy"):
            original_filename = os.path.splitext(os.path.basename(filename))[0] # Nombre del archivo original sin la extensión
            images = np.load(os.path.join(pathNumpy, carpeta, filename))
            for rotation, horizontal_flip, vertical_flip in transformations:
                output_file_path = os.path.join(f"{pathNumpySave}/{carpeta}", f'{original_filename}transformed{rotation}{horizontal_flip}{vertical_flip}.npy') 
                if not os.path.exists(output_file_path):
                    transformed_images = []
                    for image in images:
                        transformed_image = image.copy()
                        transformed_image = rotate_image_90(transformed_image, times=rotation)
                        if horizontal_flip:
                            transformed_image = flip_image_horizontal(transformed_image)
                        if vertical_flip:
                            transformed_image = flip_image_vertical(transformed_image)
                        transformed_images.append(transformed_image)
                    
                    np.save(output_file_path, np.array(transformed_images))
        print(f"Calculando: {filename} con shape {transformed_images[0].shape}               ", end='\r')

In [5]:
# Training:
pathNumpy = "E:/Numpys/Array/Bloques 607 Benign malign/Division/Training"
pathNumpySave = "E:/Numpys/Array/Bloques 607 Benign malign/DataAugmentation/Training"

transformations = [
    (0, 0, 0),  # No transformation
    (1, 0, 0),  # Rotación de 90 grados
    (2, 0, 0),  # Rotación de 180 grados
    (3, 0, 0),  # Rotación de 270 grados
    (0, 1, 0),  # Volteo horizontal
    (0, 0, 1),  # Volteo vertical
    (1, 1, 0),  # Rotación de 90 grados + Volteo horizontal
    (1, 0, 1),  # Rotación de 90 grados + Volteo vertical
    (2, 1, 0),  # Rotación de 180 grados + Volteo horizontal
    (2, 0, 1),  # Rotación de 180 grados + Volteo vertical
    (3, 1, 0),  # Rotación de 270 grados + Volteo horizontal
    (3, 0, 1),  # Rotación de 270 grados + Volteo vertical
]

contenido = os.listdir(pathNumpy)
carpetas = [elemento for elemento in contenido if os.path.isdir(os.path.join(pathNumpy, elemento))]
for carpeta in carpetas:
    for filename in os.listdir(os.path.join(pathNumpy,carpeta)):
        if filename.endswith(".npy"):
            original_filename = os.path.splitext(os.path.basename(filename))[0] # Nombre del archivo original sin la extensión
            images = np.load(os.path.join(pathNumpy, carpeta, filename))
            for rotation, horizontal_flip, vertical_flip in transformations:
                output_file_path = os.path.join(f"{pathNumpySave}/{carpeta}", f'{original_filename}transformed{rotation}{horizontal_flip}{vertical_flip}.npy') 
                if not os.path.exists(output_file_path):
                    transformed_images = []
                    for image in images:
                        transformed_image = image.copy()
                        transformed_image = rotate_image_90(transformed_image, times=rotation)
                        if horizontal_flip:
                            transformed_image = flip_image_horizontal(transformed_image)
                        if vertical_flip:
                            transformed_image = flip_image_vertical(transformed_image)
                        transformed_images.append(transformed_image)
                    
                    np.save(output_file_path, np.array(transformed_images))
        print(f"Calculando: {filename} con shape {transformed_images[0].shape}               ", end='\r')

In [6]:
# Validation:
pathNumpy = "E:/Numpys/Array/Bloques 607 Benign malign/Division/Validation"
pathNumpySave = "E:/Numpys/Array/Bloques 607 Benign malign/DataAugmentation/Validation"

transformations = [
    (0, 0, 0),  # No transformation
    (1, 0, 0),  # Rotación de 90 grados
    (2, 0, 0),  # Rotación de 180 grados
    (3, 0, 0),  # Rotación de 270 grados
    (0, 1, 0),  # Volteo horizontal
    (0, 0, 1),  # Volteo vertical
    (1, 1, 0),  # Rotación de 90 grados + Volteo horizontal
    (1, 0, 1),  # Rotación de 90 grados + Volteo vertical
    (2, 1, 0),  # Rotación de 180 grados + Volteo horizontal
    (2, 0, 1),  # Rotación de 180 grados + Volteo vertical
    (3, 1, 0),  # Rotación de 270 grados + Volteo horizontal
    (3, 0, 1),  # Rotación de 270 grados + Volteo vertical
]

contenido = os.listdir(pathNumpy)
carpetas = [elemento for elemento in contenido if os.path.isdir(os.path.join(pathNumpy, elemento))]
for carpeta in carpetas:
    for filename in os.listdir(os.path.join(pathNumpy,carpeta)):
        if filename.endswith(".npy"):
            original_filename = os.path.splitext(os.path.basename(filename))[0] # Nombre del archivo original sin la extensión
            images = np.load(os.path.join(pathNumpy, carpeta, filename))
            for rotation, horizontal_flip, vertical_flip in transformations:
                output_file_path = os.path.join(f"{pathNumpySave}/{carpeta}", f'{original_filename}transformed{rotation}{horizontal_flip}{vertical_flip}.npy') 
                if not os.path.exists(output_file_path):
                    transformed_images = []
                    for image in images:
                        transformed_image = image.copy()
                        transformed_image = rotate_image_90(transformed_image, times=rotation)
                        if horizontal_flip:
                            transformed_image = flip_image_horizontal(transformed_image)
                        if vertical_flip:
                            transformed_image = flip_image_vertical(transformed_image)
                        transformed_images.append(transformed_image)
                    
                    np.save(output_file_path, np.array(transformed_images))
        print(f"Calculando: {filename} con shape {transformed_images[0].shape}               ", end='\r')